<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFEpostProcessingRAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Installation
---



In [3]:
!nvidia-smi # allocation must be Tesla T4, P4, or P100

Tue Jul 28 21:03:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Core(s) per socket:  2
Thread(s) per core:  2


In [5]:
%%time
%%capture
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

CPU times: user 673 ms, sys: 124 ms, total: 796 ms
Wall time: 7min 20s


In [6]:
%%time
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
!pip install memory_profiler
# install parquet libraries
!pip install git+https://github.com/dask/fastparquet
!pip install python-snappy
!pip install pyarrow 

CPU times: user 605 ms, sys: 177 ms, total: 782 ms
Wall time: 5min 58s


In [7]:
from pathlib import Path
import os
from datetime import date

In [8]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import dask.dataframe as dd

In [9]:
import cudf
import dask_cudf

#Post-processing process 

1.   Load and concatenate parquet files into dask_cudf
2.   Count 'Source"
1.   Merge with Shapefile (gdf)
1.   Create output data









In [10]:
# Paths
pathShapefile = Path('/content/drive/My Drive/Colab Notebooks/01_Repository/FFE/data/shapefile')
pathParquets = Path('/content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison')

os.listdir(pathShapefile)

['buildings_raw.cpg',
 'buildings_raw.dbf',
 'buildings_raw.prj',
 'buildings_raw.qpj',
 'buildings_raw.shp',
 'buildings_raw.shx',
 'WellWHV_Buildings.prj',
 'WellWHV_Buildings.cpg',
 'WellWHV_Buildings.shp',
 'WellWHV_Buildings.shx',
 'WellWHV_Buildings.dbf',
 'WellWHV_Buildings.sbn',
 'WellWHV_Buildings.sbx',
 'WellWHV_Buildings.shp.xml']

In [12]:
def read_and_concatenate_parquets(path=pathParquets):
  L = []
  files = pathParquets.glob('*.parquet')
  number = 0
  for file in files:
    number += 1
    print(f"file loaded : {number}/{len(os.listdir(pathParquets))}")
    pqt = dd.read_parquet(file, engine='auto')
    # pqt = pd.read_parquet(file, columns=['source'])#, engine='auto')
    # pqtCudfDask = dask_cudf.from_dask_dataframe(pqt)
    L.append(pqt)
  df = dd.concat(L)
  return df

def count_fid_occurences(df):
  count = df['source'].value_counts().compute()
  count_df = pd.DataFrame({'source': count.index, 'count': count.values})
  count_df.to_parquet(str(pathShapefile) + '/' + f'CountBurn-{str(date.today())}).parquet', engine='auto', compression="GZIP")# could be datetime.now
  return count_df

def Merge(countDf,nameShapefile):
  # Shapefile
  gdfShape = gpd.read_file(pathShapefile / nameShapefile)
  gdfShape.insert(0, 'FID', range(0, len(gdf)))
  gdfShape.rename(columns={'FID': 'source'}, inplace=True)
  gdfShape = gdfShape[['source', 'geometry']]
  # gdf = pd.DataFrame(gdfShape)
  # cudf_gdf = cudf.DataFrame.from_pandas(gdf)
  # cudf_gdf.rename(columns={'FID': 'source'}, inplace=True)
  # cudf_gdf = cudf_gdf[['source', 'geometry']]
  # countDf as cudf
  # cudf_countDf = cudf.DataFrame.from_pandas(countDf)
  # merge
  merged = countDf.merge(gdfShape, on=['source'], how='left')
  return merged

def createShapefile(df):
  # df = df.to_pandas()
  gdf = gpd.GeoDataFrame(df, geometry='geometry')
  # gdf.plot(column='count', cmap='hsv', legend=True)
  gdf.to_file(os.path.join(str(pathShapefile) + "/" + "Burn3000scenarioWellington.shp"))
  return gdf

In [ ]:
%%time
concatDf = read_and_concatenate_parquets()
countConcatDf = count_fid_occurences(concatDf)

file loaded : 567/2981
file loaded : 568/2981
file loaded : 569/2981
file loaded : 570/2981
file loaded : 571/2981
file loaded : 572/2981
file loaded : 573/2981
file loaded : 574/2981
file loaded : 575/2981
file loaded : 576/2981
file loaded : 577/2981
file loaded : 578/2981
file loaded : 579/2981
file loaded : 580/2981
file loaded : 581/2981
file loaded : 582/2981
file loaded : 583/2981
file loaded : 584/2981
file loaded : 585/2981
file loaded : 586/2981
file loaded : 587/2981
file loaded : 588/2981
file loaded : 589/2981
file loaded : 590/2981
file loaded : 591/2981
file loaded : 592/2981
file loaded : 593/2981
file loaded : 594/2981
file loaded : 595/2981
file loaded : 596/2981
file loaded : 597/2981
file loaded : 598/2981
file loaded : 599/2981
file loaded : 600/2981
file loaded : 601/2981
file loaded : 602/2981
file loaded : 603/2981
file loaded : 604/2981
file loaded : 605/2981
file loaded : 606/2981
file loaded : 607/2981
file loaded : 608/2981
file loaded : 609/2981
file loaded

In [ ]:
mergedDf = Merge(countConcatDf, 'WellWHV_Buildings.shp')
countShape = createShapefile(df)

In [ ]:
import matplotlib as plt
import contextily as ctx
%matplotlib inline
plt.style.use('seaborn-whitegrid')

originalShape = gpd.read_file(pathShapefile / 'WellWHV_Buildings.shp')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 10))

p1 = originalShape.plot(ax=ax1, column='IgnProbBld', cmap='hsv', alpha=0.7, legend=True)
ctx.add_basemap(ax1, crs=2193)
p2 = countShape.plot(ax=ax2, column='count', cmap='YlOrRd', alpha=0.7, legend=True)
ctx.add_basemap(ax2, crs=2193)

ax1.set_title('Original Shapefile - Ignition Probability')
ax1.ticklabel_format(useOffset=False, style='plain')
ax2.set_title('Burn Count')
ax2.ticklabel_format(useOffset=False, style='plain')
ax1.tick_params(direction='out', length=6)
ax2.tick_params(direction="out", length=6)

fig.autofmt_xdate()
plt.tight_layout()
plt.show()